<a href="https://colab.research.google.com/github/paulobitar/posfasam-bootcamppython/blob/main/scrap_anac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import json
import urllib
import requests
import io
import pandas as pd
from bs4 import BeautifulSoup

In [40]:
#recuperando o número de serie dos dados abertos da ANAC
#limitado a 10 linhas por uma questão de performance
url = "https://www.anac.gov.br/acesso-a-informacao/dados-abertos/areas-de-atuacao/aeronaves/registro-aeronautico-brasileiro/aeronaves-registradas-no-registro-aeronautico-brasileiro-csv"
df_num_serie = pd.read_csv(url, sep=';', skiprows=1, encoding ='latin1', usecols=['NUM_SERIE'], nrows=10)


In [ ]:
url = 'https://sistemas.anac.gov.br/aeronaves/cons_rab_resposta.asp?textMarca=&selectHabilitacao=&selectIcao=&selectModelo=&selectFabricante=&textNumeroSerie='
lista_aeronave=[]

for index, row in df_num_serie.iterrows():
  #gerando a URL de Request com o número de série
  r  = requests.get(url+row['NUM_SERIE'])
  data = r.text
  soup = BeautifulSoup(data,"html.parser")
  #localizando a tabela onde estão os dados
  tabela = soup.find("table", {"class": "table table-hover"})
  #usando o método read_html do pŕoprio pandas para transformar a tabela em um Dataframe
  df = pd.read_html(str(tabela))[0]
  df.rename(columns={0:'campo',1:'valor'}, inplace=True)
  #ZIP retorna uma lista de tuplas, onde cada linha i-ésima tupla contém o i-ésimo elemento de cada um dos argumentos.
  #Assim as linhas da coluna CAMPO se transformam em Colunas para cada valor
  lista_aeronave.append(dict(zip(df.campo, df.valor)))
  


In [58]:
#Transformando a lista em Dataframe
df_aeronave = pd.DataFrame(lista_aeronave)
#Os campos estão terminando com : . Tratamento para retirar isso usando list comprehension
temp_cols = df_aeronave.columns
final_cols = [x.replace(':','') for x in temp_cols]
df_aeronave.columns = [x.replace(':','').strip() for x in temp_cols]

print(df_aeronave.to_string())

                       Proprietário        CPF/CNPJ                         Operador                        Fabricante Ano de Fabricação           Modelo Número de Série Tipo ICAO Tipo de Habilitação para Pilotos                         Classe da Aeronave Peso Máximo de Decolagem Número Máximo de Passageiros Tipo de voo autorizado           Categoria de Registro Número da Matrícula               Status da Operação  Outros Proprietarios
0              GOVERNO FEDERAL ANAC  18607499000165            AEROCLUBE DE VARGINHA                               IPE              1989          IPE-02B             063      GLID                             PLAN              POUSO CONVECIONAL 0 SEM MOTOR                 565 - Kg                          001             VFR Diurno               PRIVADA INSTRUCAO               12394  OPERAÇÃO NEGADA PARA TÁXI AÉREO                   NaN
1      LUIZ FERNANDO SEGADAS VIANNA     28866983772     LUIZ FERNANDO SEGADAS VIANNA      LUIZ FERNANDO SEGADAS VIANNA  